In [1]:
%load_ext autoreload
%autoreload 2

import os
import re
import math
import glob
from importlib import reload

import tqdm 
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import osu.rulesets.beatmap as bm
import osu.rulesets.replay as rp
import osu.rulesets.hitobjects as hitobjects
import osu.dataset as dataset

import osu.preview.preview as preview

osu! path: C:\Users\niooi\AppData\Roaming/../Local/osu!
pygame 2.6.1 (SDL 2.28.4, Python 3.12.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


E:\Projects\osu\.venv\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
obj_dataset = dataset.replay_mapping_from_cache(2000)

Loading objects from .data/: 100%|██████████| 2000/2000 [00:44<00:00, 44.99it/s]


In [28]:
obj_dataset

,replay,beatmap
0,<osu.rulesets.replay.Replay object at 0x000001...,<osu.rulesets.beatmap.Beatmap object at 0x0000...
1,<osu.rulesets.replay.Replay object at 0x000001...,<osu.rulesets.beatmap.Beatmap object at 0x0000...
2,<osu.rulesets.replay.Replay object at 0x000001...,<osu.rulesets.beatmap.Beatmap object at 0x0000...
3,<osu.rulesets.replay.Replay object at 0x000001...,<osu.rulesets.beatmap.Beatmap object at 0x0000...
4,<osu.rulesets.replay.Replay object at 0x000001...,<osu.rulesets.beatmap.Beatmap object at 0x0000...
...,...,...
1995,<osu.rulesets.replay.Replay object at 0x000001...,<osu.rulesets.beatmap.Beatmap object at 0x0000...
1996,<osu.rulesets.replay.Replay object at 0x000001...,<osu.rulesets.beatmap.Beatmap object at 0x0000...
1997,<osu.rulesets.replay.Replay object at 0x000001...,<osu.rulesets.beatmap.Beatmap object at 0x0000...
1998,<osu.rulesets.replay.Replay object at 0x000001...,<osu.rulesets.beatmap.Beatmap object at 0x0000...


In [ ]:
input_data = dataset.input_data(obj_dataset, verbose=True)
output_data = dataset.target_data(obj_dataset, verbose=True)

Turning beatmaps into time series data:  12%|█▏        | 234/2000 [02:22<23:58,  1.23it/s]

In [ ]:
input = input_data
output = output_data

In [ ]:
input[0:500]

In [ ]:
output[0:500]

In [ ]:
import torch

xs = np.reshape(input.values, (-1, dataset.BATCH_LENGTH, len(dataset.INPUT_FEATURES)))

# try:
#     target_data = pd.read_pickle('.data/target_data.dat')
# except:
#     target_data = dataset.target_data(dataset, verbose=True)
#     target_data.to_pickle('.data/target_data.dat')

ys = np.reshape(output.values, (-1, dataset.BATCH_LENGTH, len(dataset.OUTPUT_FEATURES)))

xs.shape

In [ ]:
ys.shape

In [ ]:
# save dataset so we have it later

# os.makedirs(f'.datasets', exist_ok=True)
# 
# np.save(f'.datasets/xs.npy', xs)
# np.save(f'.datasets/ys.npy', ys)

In [2]:
# reload from saved

xs = np.load(f'.datasets/xs.npy')
ys = np.load(f'.datasets/ys.npy')

In [3]:
# tys = ys.reshape(ys.shape[0] * ys.shape[1], ys.shape[2])
# for frame in tys:
#     if np.isinf(frame).any():
#         print(frame)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from random import randint 
from torch.utils.data import DataLoader, TensorDataset

BATCH_SIZE = 128

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 
# x_train, x_test, y_train, y_test = train_test_split(xs, ys, test_size=0.2, random_state=randint(0, 100))
# 
# train_dataset = TensorDataset(torch.FloatTensor(x_train), torch.FloatTensor(y_train))
# 
# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [10]:
from osu.rnn import OsuReplayRNN

In [11]:
print("Creating RNN model...")
rnn = OsuReplayRNN(batch_size=BATCH_SIZE, noise_std=0.00)
rnn.load_data(xs, ys)

Creating RNN model...
Position model compiled with torch.compile
RNN Model initialized on cuda (noise_std=0.0)
Position model parameters: 227650
Data loaded: 9073 training samples, 2269 test samples (position only)


In [12]:
RNN_EPOCHS = 25

# Train the RNN
rnn.train(epochs=RNN_EPOCHS)
rnn.plot_losses()
rnn.save()

Epoch 1/25 (Train): 100%|██████████| 71/71 [00:07<00:00,  9.02it/s]


Epoch 1/25, Train: 0.0072, Test: 0.0022


Epoch 2/25 (Train): 100%|██████████| 71/71 [00:07<00:00,  9.46it/s]


Epoch 2/25, Train: 0.0021, Test: 0.0018


Epoch 3/25 (Train): 100%|██████████| 71/71 [00:07<00:00,  9.68it/s]


...

Epoch 16/25, Train: 0.0012, Test: 0.0011


Epoch 17/25 (Train): 100%|██████████| 71/71 [00:07<00:00,  9.43it/s]


Epoch 17/25, Train: 0.0012, Test: 0.0011


Epoch 18/25 (Train): 100%|██████████| 71/71 [00:07<00:00,  9.64it/s]


KeyboardInterrupt: 

In [9]:
# import torch
# import gc
# 
# # Clear GPU cache
# torch.cuda.empty_cache()
# 
# # Force garbage collection
# gc.collect()
# 
# del rnn

NameError: name 'rnn' is not defined

In [ ]:
from osu.gan import OsuReplayGAN

In [ ]:
print("Creating GAN model...")
gan = OsuReplayGAN(batch_size=BATCH_SIZE)
gan.load_data(xs, ys)

In [ ]:
GAN_EPOCHS = 4

# Train the GAN
for i in range(8):
    gan.train(epochs=GAN_EPOCHS)
    gan.save()
    
gan.plot_losses()

In [ ]:
from osu.keys import OsuKeyModel

In [ ]:
print("Creating keypress model...")
keys = OsuKeyModel(batch_size=BATCH_SIZE)
keys.load_data(xs, ys)

In [ ]:
KEYS_EPOCHS = 12

# Train the key model
for i in range(12):
    keys.train(epochs=KEYS_EPOCHS)
    keys.save()
    
keys.plot_losses()

In [ ]:
del keys

In [22]:
from osu.rulesets.mods import Mods
import osu.rulesets.beatmap as bm
import osu.rulesets.replay as rp
import osu.dataset as dataset

test_name = '1hope'
test_mods = Mods.HARD_ROCK
test_map_path = f'assets/{test_name}_map.osu'
test_song = f'assets/{test_name}_song.mp3'

test_map = bm.load(test_map_path)
test_map.apply_mods(test_mods)

data = dataset.input_data(test_map)
data = np.reshape(data.values, (-1, dataset.BATCH_LENGTH, len(dataset.INPUT_FEATURES)))
data = torch.FloatTensor(data)

Turning beatmaps into time series data: 100%|██████████| 1/1 [00:00<00:00,  1.18it/s]


In [23]:
with torch.no_grad():
    # Use generator for inference
    replay_data = rnn.generate(data)
    
replay_data = np.concatenate(replay_data)
replay_data = np.pad(replay_data, ((0, 0), (0, 2)), mode='constant', constant_values=0)
if not os.path.exists('.generated'):
    os.makedirs('.generated')
    
np.save('.generated\\' + test_name + '.npy', replay_data)

print(f"Generated replay data shape: {replay_data.shape}")
print(f"Saved to .generated\\{test_name}.npy")

Generated replay data shape: (10240, 4)
Saved to .generated\1hope.npy


In [24]:
data

tensor([[[ 0.0684, -0.3724,  0.4500,  ...,  0.5200,  0.0000,  0.0000],
         [ 0.0684, -0.3724,  0.4260,  ...,  0.5200,  0.0000,  0.0000],
         [ 0.0684, -0.3724,  0.4020,  ...,  0.5200,  0.0000,  0.0000],
         ...,
         [-0.3262,  0.3880,  0.0940,  ...,  0.5200,  1.0260,  0.3733],
         [-0.3262,  0.3880,  0.0700,  ...,  0.5200,  1.0260,  0.3733],
         [-0.3262,  0.3880,  0.0460,  ...,  0.5200,  1.0260,  0.3733]],

        [[-0.3262,  0.3880,  0.0220,  ...,  0.5200,  1.0260,  0.3733],
         [-0.3242,  0.3880,  0.0000,  ...,  0.5200,  1.0260,  0.3733],
         [-0.2812,  0.4167,  0.0000,  ...,  0.5200,  1.0260,  0.3733],
         ...,
         [-0.1016, -0.3672,  0.0930,  ...,  0.5200,  0.0000,  0.0000],
         [-0.1016, -0.3672,  0.0690,  ...,  0.5200,  0.0000,  0.0000],
         [-0.1016, -0.3672,  0.0450,  ...,  0.5200,  0.0000,  0.0000]],

        [[-0.1016, -0.3672,  0.0210,  ...,  0.5200,  0.0000,  0.0000],
         [-0.3145,  0.0651,  0.1070,  ...,  0

In [25]:
replay_data[:500]

array([[-0.03782767, -0.0210133 ,  0.        ,  0.        ],
       [-0.04166194, -0.01667719,  0.        ,  0.        ],
       [-0.04060125, -0.02386662,  0.        ,  0.        ],
       ...,
       [ 0.02833094,  0.27376795,  0.        ,  0.        ],
       [ 0.07345363,  0.25934333,  0.        ,  0.        ],
       [ 0.10369711,  0.25983548,  0.        ,  0.        ]],
      shape=(500, 4), dtype=float32)

In [26]:
import importlib
import osu.preview.preview as preview

importlib.reload(preview)

preview.preview_replay_raw(replay_data, test_map_path, test_mods, test_song)